<a href="https://colab.research.google.com/github/flydragon2018/colab_notebooks/blob/main/colab_yolox_reef_cots_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/mydrive', force_remount=True)

Mounted at /content/mydrive


In [2]:
#!pip -q install fsspec
#!pip -q  install gcsfs

     |████████████████████████████████| 132 kB 4.0 MB/s 
     |████████████████████████████████| 1.1 MB 6.1 MB/s 
     |████████████████████████████████| 160 kB 50.3 MB/s 
     |████████████████████████████████| 271 kB 50.8 MB/s 
     |████████████████████████████████| 192 kB 48.8 MB/s 


In [3]:
#import gcsfs
#fs = gcsfs.GCSFileSystem()

In [4]:
##!gsutil -m cp -r  gs://kds-87f39ccfe721b0f0b4087508aefd658220bc65ce44e62df765a13734/  /content/mydrive/MyDrive/kaggle/

In [2]:
import warnings
warnings.filterwarnings("ignore")

import ast
import os
import json
import pandas as pd
import torch
import importlib
import cv2 

from shutil import copyfile
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import GroupKFold
from PIL import Image
from string import Template
from IPython.display import display


In [3]:

TRAIN_PATH = '/content/mydrive/MyDrive/kaggle/reef'

In [4]:
# check Torch and CUDA version
print(f"Torch: {torch.__version__}")
!nvcc --version

Torch: 1.10.0+cu111
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [5]:
!git clone https://github.com/Megvii-BaseDetection/YOLOX -q

%cd YOLOX
!pip install -U pip && pip install -r requirements.txt
!pip install -v -e . 

/content/YOLOX
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 57 kB 2.9 MB/s             
     |████████████████████████████████| 108 kB 11.1 MB/s            
     |████████████████████████████████| 14.5 MB 17.0 MB/s            
     |████████████████████████████████| 4.5 MB 41.3 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 466 kB 43.5 MB/s            
  Created wheel for onnx-simplifier: filename=onnx_simplifier-0.3.5-py3-none-any.whl size=12878 sha256=be813af1019d99ea7507915d3b674e67ede6c43600f839886159e0be7fce7af3
  Stored in directory: /root/.cache/pip/wheels/8a/b4/1b/6acdd4eb854b215cd4aa1c18ca79399f9d34728edaff47ecce
Successfully built onnx-simplifier
Using pip 21.3.1 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Obtaining f

In [6]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_path(row):
    row['image_path'] = f'{TRAIN_PATH}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

In [7]:
df = pd.read_csv("/content/mydrive/MyDrive/kaggle/reef/train.csv")
df.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [8]:
# Taken only annotated photos
df["num_bbox"] = df['annotations'].apply(lambda x: str.count(x, 'x'))
df_train = df[df["num_bbox"]>0]

#Annotations 
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

#Images resolution
df_train["width"] = 1280
df_train["height"] = 720

#Path of images
df_train = df_train.progress_apply(get_path, axis=1)

  0%|          | 0/4919 [00:00<?, ?it/s]

  0%|          | 0/4919 [00:00<?, ?it/s]

  0%|          | 0/4919 [00:00<?, ?it/s]

In [9]:
kf = GroupKFold(n_splits = 5) 
df_train = df_train.reset_index(drop=True)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y = df_train.video_id.tolist(), groups=df_train.sequence)):
    df_train.loc[val_idx, 'fold'] = fold

df_train.head(5)

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,num_bbox,bboxes,width,height,image_path,fold
0,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1,"[[559, 213, 50, 32]]",1280,720,/content/mydrive/MyDrive/kaggle/reef/train_ima...,4
1,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1,"[[558, 213, 50, 32]]",1280,720,/content/mydrive/MyDrive/kaggle/reef/train_ima...,4
2,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",1,"[[557, 213, 50, 32]]",1280,720,/content/mydrive/MyDrive/kaggle/reef/train_ima...,4
3,0,40258,19,19,0-19,"[{'x': 556, 'y': 214, 'width': 50, 'height': 32}]",1,"[[556, 214, 50, 32]]",1280,720,/content/mydrive/MyDrive/kaggle/reef/train_ima...,4
4,0,40258,20,20,0-20,"[{'x': 555, 'y': 214, 'width': 50, 'height': 32}]",1,"[[555, 214, 50, 32]]",1280,720,/content/mydrive/MyDrive/kaggle/reef/train_ima...,4


In [10]:
SELECTED_FOLD = 4

In [15]:
HOME_DIR = '/content/mydrive/MyDrive/kaggle/reefdata/' 
DATASET_PATH = 'dataset{}/images'.format(SELECTED_FOLD)

#!mkdir {HOME_DIR}dataset
!mkdir -p {HOME_DIR}{DATASET_PATH}
!mkdir -p {HOME_DIR}{DATASET_PATH}/train2017
!mkdir -p {HOME_DIR}{DATASET_PATH}/val2017
!mkdir -p {HOME_DIR}{DATASET_PATH}/annotations

In [16]:


for i in tqdm(range(len(df_train))):
    row = df_train.loc[i]
    if row.fold != SELECTED_FOLD:
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/train2017/{row.image_id}.jpg')
    else:
        copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/val2017/{row.image_id}.jpg') 

  0%|          | 0/4919 [00:00<?, ?it/s]

In [17]:
print(f'Number of training files: {len(os.listdir(f"{HOME_DIR}{DATASET_PATH}/train2017/"))}')
print(f'Number of validation files: {len(os.listdir(f"{HOME_DIR}{DATASET_PATH}/val2017/"))}')

Number of training files: 3974
Number of validation files: 945


In [18]:
def save_annot_json(json_annotation, filename):
    with open(filename, 'w') as f:
        output_json = json.dumps(json_annotation)
        f.write(output_json)

In [19]:
annotion_id = 0

In [20]:
def dataset2coco(df, dest_path):
    
    global annotion_id
    
    annotations_json = {
        "info": [],
        "licenses": [],
        "categories": [],
        "images": [],
        "annotations": []
    }
    
    info = {
        "year": "2021",
        "version": "1",
        "description": "COTS dataset - COCO format",
        "contributor": "",
        "url": "https://kaggle.com",
        "date_created": "2021-11-30T15:01:26+00:00"
    }
    annotations_json["info"].append(info)
    
    lic = {
            "id": 1,
            "url": "",
            "name": "Unknown"
        }
    annotations_json["licenses"].append(lic)

    classes = {"id": 0, "name": "starfish", "supercategory": "none"}

    annotations_json["categories"].append(classes)

    
    for ann_row in df.itertuples():
            
        images = {
            "id": ann_row[0],
            "license": 1,
            "file_name": ann_row.image_id + '.jpg',
            "height": ann_row.height,
            "width": ann_row.width,
            "date_captured": "2021-11-30T15:01:26+00:00"
        }
        
        annotations_json["images"].append(images)
        
        bbox_list = ann_row.bboxes
        
        for bbox in bbox_list:
            b_width = bbox[2]
            b_height = bbox[3]
            
            # some boxes in COTS are outside the image height and width
            if (bbox[0] + bbox[2] > 1280):
                b_width = bbox[0] - 1280 
            if (bbox[1] + bbox[3] > 720):
                b_height = bbox[1] - 720 
                
            image_annotations = {
                "id": annotion_id,
                "image_id": ann_row[0],
                "category_id": 0,
                "bbox": [bbox[0], bbox[1], b_width, b_height],
                "area": bbox[2] * bbox[3],
                "segmentation": [],
                "iscrowd": 0
            }
            
            annotion_id += 1
            annotations_json["annotations"].append(image_annotations)
        
        
    print(f"Dataset COTS annotation to COCO json format completed! Files: {len(df)}")
    return annotations_json


In [21]:
# Convert COTS dataset to JSON COCO
train_annot_json = dataset2coco(df_train[df_train.fold != SELECTED_FOLD], f"{HOME_DIR}{DATASET_PATH}/train2017/")
val_annot_json = dataset2coco(df_train[df_train.fold == SELECTED_FOLD], f"{HOME_DIR}{DATASET_PATH}/val2017/")

# Save converted annotations
save_annot_json(train_annot_json, f"{HOME_DIR}{DATASET_PATH}/annotations/train.json")
save_annot_json(val_annot_json, f"{HOME_DIR}{DATASET_PATH}/annotations/valid.json")

Dataset COTS annotation to COCO json format completed! Files: 3974
Dataset COTS annotation to COCO json format completed! Files: 945


In [22]:
# Choose model for your experiments NANO or YOLOX-S (you can adapt for other model type)

NANO = False

In [50]:
config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp

SELECTED_FOLD = 4

class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        #self.depth = 0.33
        #self.width = 0.50
        self.depth = 1.00
        self.width = 1.00
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        
        # Define yourself dataset path
        #self.data_dir = "/kaggle/working/dataset/images"
        self.data_dir = "/content/mydrive/MyDrive/kaggle/reefdata/dataset{}/images".format(SELECTED_FOLD)
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1
        
        self.mosaic_prob = 1.0
        self.mixup_prob = 1.0
        self.hsv_prob = 1.0
        self.flip_prob = 0.5
        self.no_aug_epochs = 2
        
        self.input_size = (960, 960)
        self.mosaic_scale = (0.5, 1.5)
        self.random_size = (10, 20)
        self.test_size = (960, 960)
'''

In [39]:
if NANO:
    config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

import torch.nn as nn

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.33
        self.width = 0.25
        self.input_size = (416, 416)
        self.mosaic_scale = (0.5, 1.5)
        self.random_size = (10, 20)
        self.test_size = (416, 416)
        self.exp_name = os.path.split(
            os.path.realpath(__file__))[1].split(".")[0]
        self.enable_mixup = False

        # Define yourself dataset path
        self.data_dir = "/kaggle/working/dataset/images"
        self.train_ann = "train.json"
        self.val_ann = "valid.json"

        self.num_classes = 1

        self.max_epoch = $max_epoch
        self.data_num_workers = 2
        self.eval_interval = 1

    def get_model(self, sublinear=False):
        def init_yolo(M):
            for m in M.modules():
                if isinstance(m, nn.BatchNorm2d):
                    m.eps = 1e-3
                    m.momentum = 0.03

        if "model" not in self.__dict__:
            from yolox.models import YOLOX, YOLOPAFPN, YOLOXHead
            in_channels = [256, 512, 1024]
            # NANO model use depthwise = True, which is main difference.
            backbone = YOLOPAFPN(self.depth,
                                 self.width,
                                 in_channels=in_channels,
                                 depthwise=True)
            head = YOLOXHead(self.num_classes,
                             self.width,
                             in_channels=in_channels,
                             depthwise=True)
            self.model = YOLOX(backbone, head)

        self.model.apply(init_yolo)
        self.model.head.initialize_biases(1e-2)
        return self.model

'''

In [51]:
PIPELINE_CONFIG_PATH='cots_config.py'

#pipeline = Template(config_file_template).substitute(max_epoch = 20)
pipeline = Template(config_file_template).substitute(max_epoch = 60)

with open(PIPELINE_CONFIG_PATH, 'w') as f:
    f.write(pipeline)

In [52]:
# ./yolox/data/datasets/voc_classes.py

voc_cls = '''
VOC_CLASSES = (
  "starfish",
)
'''
with open('./yolox/data/datasets/voc_classes.py', 'w') as f:
    f.write(voc_cls)

# ./yolox/data/datasets/coco_classes.py

coco_cls = '''
COCO_CLASSES = (
  "starfish",
)
'''
with open('./yolox/data/datasets/coco_classes.py', 'w') as f:
    f.write(coco_cls)

# check if everything is ok    
!more ./yolox/data/datasets/coco_classes.py



COCO_CLASSES = (
  "starfish",
)


In [42]:
#sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_s.pth'
#MODEL_FILE = 'yolox_s.pth'

'''
sh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_l.pth'
MODEL_FILE = 'yolox_l.pth'

if NANO:
    sh = ''
    wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth
    ''
    MODEL_FILE = 'yolox_nano.pth'

with open('script.sh', 'w') as file:
  file.write(sh)

!bash script.sh

'''


"\nsh = 'wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_l.pth'\nMODEL_FILE = 'yolox_l.pth'\n\nif NANO:\n    sh = ''\n    wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pth\n    ''\n    MODEL_FILE = 'yolox_nano.pth'\n\nwith open('script.sh', 'w') as file:\n  file.write(sh)\n\n!bash script.sh\n\n"

In [43]:
!cp ./tools/train.py ./

In [44]:
!unzip -q /content/mydrive/MyDrive/kaggle/reef/yx_l_003.pth.zip   

replace yx_l_003.pth? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [45]:
 '''
!python train.py \
    -f cots_config.py \
    -d 1 \
    -b 8 \
    --fp16 \
    -o \
    -c /content/YOLOX/yolox_l.pth 
 '''

'\n!python train.py    -f cots_config.py    -d 1    -b 8    --fp16    -o    -c /content/YOLOX/yolox_l.pth \n'

In [ ]:
!python train.py \
    -f cots_config.py \
    -d 1 \
    -b 4 \
    --fp16 \
    -o \
    -c /content/YOLOX/yx_l_003.pth   # Remember to chenge this line if you take different model eg. yolo_nano.pth, yolox_s.pth or yolox_m.pth

2021-12-08 07:29:52 | INFO     | yolox.core.trainer:126 - args: Namespace(batch_size=4, cache=False, ckpt='/content/YOLOX/yx_l_003.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='cots_config.py', experiment_name='cots_config', fp16=True, machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2021-12-08 07:29:52 | INFO     | yolox.core.trainer:127 - exp value:
╒══════════════════╤════════════════════════════════════════════════════════════╕
│ keys             │ values                                                     │
╞══════════════════╪════════════════════════════════════════════════════════════╡
│ seed             │ None                                                       │
├──────────────────┼────────────────────────────────────────────────────────────┤
│ output_dir       │ './YOLOX_outputs'                                          │
├──────────────────┼────────────────────────────────────────────────────────────┤
│ print

In [47]:
# I have to fix demo.py file because it:
# - raises error in Kaggle (cvWaitKey does not work) 
# - saves result files in time named directory eg. /2021_11_29_22_51_08/ which is difficult then to automatically show results

%cp /content/mydrive/MyDrive/kaggle/reef/demo.py   tools/demo.py

In [48]:
TEST_IMAGE_PATH = "/content/mydrive/MyDrive/kaggle/reefdata/dataset/images/train2017/0-2331.jpg"
MODEL_PATH = "./YOLOX_outputs/cots_config/best_ckpt.pth"

!python tools/demo.py image \
    -f cots_config.py \
    -c {MODEL_PATH} \
    --path {TEST_IMAGE_PATH} \
    --conf 0.1 \
    --nms 0.45 \
    --tsize 960 \
    --save_result \
    --device gpu

Traceback (most recent call last):
  File "/content/YOLOX/yolox/exp/build.py", line 14, in get_exp_by_file
    exp = current_exp.Exp()
  File "/content/YOLOX/cots_config.py", line 23, in __init__
    self.data_dir = "/content/mydrive/MyDrive/kaggle/reefdata/dataset{}/images".format(SELECTED_FOLD)
NameError: name 'SELECTED_FOLD' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "tools/demo.py", line 322, in <module>
    exp = get_exp(args.exp_file, args.name)
  File "/content/YOLOX/yolox/exp/build.py", line 51, in get_exp
    return get_exp_by_file(exp_file)
  File "/content/YOLOX/yolox/exp/build.py", line 16, in get_exp_by_file
    raise ImportError("{} doesn't contains class named 'Exp'".format(exp_file))
ImportError: cots_config.py doesn't contains class named 'Exp'


In [49]:
from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

COCO_CLASSES = (
  "starfish",
)

# get YOLOX experiment
current_exp = importlib.import_module('cots_config')
exp = current_exp.Exp()

# set inference parameters
test_size = (960, 960)
num_classes = 1
confthre = 0.1
nmsthre = 0.45


# get YOLOX model
model = exp.get_model()
model.cuda()
model.eval()

# get custom trained checkpoint
ckpt_file = "./YOLOX_outputs/cots_config/best_ckpt.pth"
ckpt = torch.load(ckpt_file, map_location="cpu")
model.load_state_dict(ckpt["model"])


FileNotFoundError: ignored

In [ ]:
def yolox_inference(img, model, test_size): 
    bboxes = []
    bbclasses = []
    scores = []
    
    preproc = ValTransform(legacy = False)

    tensor_img, _ = preproc(img, None, test_size)
    tensor_img = torch.from_numpy(tensor_img).unsqueeze(0)
    tensor_img = tensor_img.float()
    tensor_img = tensor_img.cuda()

    with torch.no_grad():
        outputs = model(tensor_img)
        outputs = postprocess(
                    outputs, num_classes, confthre,
                    nmsthre, class_agnostic=True
                )

    if outputs[0] is None:
        return [], [], []
    
    outputs = outputs[0].cpu()
    bboxes = outputs[:, 0:4]

    bboxes /= min(test_size[0] / img.shape[0], test_size[1] / img.shape[1])
    bbclasses = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    return bboxes, bbclasses, scores


In [ ]:
def draw_yolox_predictions(img, bboxes, scores, bbclasses, confthre, classes_dict):
    for i in range(len(bboxes)):
            box = bboxes[i]
            cls_id = int(bbclasses[i])
            score = scores[i]
            if score < confthre:
                continue
            x0 = int(box[0])
            y0 = int(box[1])
            x1 = int(box[2])
            y1 = int(box[3])

            cv2.rectangle(img, (x0, y0), (x1, y1), (0, 255, 0), 2)
            cv2.putText(img, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,255,0), thickness = 1)
    return img


In [ ]:
TEST_IMAGE_PATH = "/content/mydrive/MyDrive/kaggle/reefdata/dataset/images/val2017/0-4614.jpg"
img = cv2.imread(TEST_IMAGE_PATH)

# Get predictions
bboxes, bbclasses, scores = yolox_inference(img, model, test_size)

# Draw predictions
out_image = draw_yolox_predictions(img, bboxes, scores, bbclasses, confthre, COCO_CLASSES)

# Since we load image using OpenCV we have to convert it 
out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)
display(Image.fromarray(out_image))

In [ ]:
cp -rf /content/YOLOX/YOLOX_outputs    /content/mydrive/MyDrive/